In [6]:
import datetime as dt
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.optimize import minimize   

In [7]:
# load data
DATA_DIR = 'data'
FILE_NAMES = ['1st_wave_data_cro.csv',
              '2nd_wave_data_cro.csv']
FILE_PATHS = [os.path.join(os.pardir, DATA_DIR, FILE_NAMES[0]),
              os.path.join(os.pardir, DATA_DIR, FILE_NAMES[1])]
df_1 = pd.read_csv(FILE_PATHS[0])
df_2 = pd.read_csv(FILE_PATHS[1])
df = pd.concat([df_1, df_2[1:-1]], sort=False)

cum_positives = np.cumsum(df.confirmed_positives.values)
cum_recovered = df.estimate_recovered.values
cum_deceased = np.cumsum(df.confirmed_deceased.values)
active = cum_positives - cum_recovered - cum_deceased

# start of different epi waves
eff_dates=[dt.datetime(2020, 2, 25), dt.datetime(2020, 6, 1), dt.datetime(2020, 8, 8)]

In [8]:
def MSE(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

In [9]:
def SEIRD(t, y, beta, alpha, gamma, mu):
    S, E, I, R, D = y
    N = S + E + I + R + D
    return [
        -beta*S*I/N, 
        beta*S*I/N - alpha*E, 
        alpha*E - gamma*I - mu*I, 
        gamma*I,
        mu*I,
    ]

In [10]:
def loss(params, active, cum_recovered, cum_deceased, IC):
        size = active.size
        sol = solve_ivp(
            fun=SEIRD, 
            t_span=(0, size), 
            y0=IC, 
            args=params,
            method='RK45', 
            t_eval=np.arange(0, size, 1), 
            vectorized=True,
        )
        return MSE(sol.y[2], active)

In [11]:
guess = [0.1, 0.1, 0.1, 0.1]
S0 = 
E0 = 
I0 = 
R0 = 
D0 =
IC = [S0, E0, I0, R0, D0]

In [12]:
opt = minimize(
    fun=loss, 
    x0=guess,
    args=(active, cum_recovered, cum_deceased, IC),
    method='L-BFGS-B',
    bounds=[(1e-5, 1.0)] * len(guess),
    options={'maxiter': 1000, 'disp': True},
)

NameError: name 'IC' is not defined